<a href="https://colab.research.google.com/github/danadler-dev/mesa_examples/blob/main/mesa_ff_potatohead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An example of a model embedded in a grid - based on the [Mesa](https://mesa.readthedocs.io) Agent Based Modeling library. The idea behind this example is explained in my article: [Object-Oriented or Functional Morphogenetic Code:
From The French Flag to Mr. Potato Head](https://colab.research.google.com/github/danadler-dev/OO-morpho-genetic-code/blob/main/FrenchFlagToMrPotatoHead.ipynb) found in this [github repo](https://github.com/danadler-dev/OO-morpho-genetic-code).

In [ ]:
!pip install mesa

In [20]:
import numpy as np
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual, IntSlider
from matplotlib import animation, rc
from IPython.display import HTML
import math

The French Flag Mesa model (see example reference above). 

In [26]:
def rect(top_left, bot_right, rgb):
  return lambda x, y: rgb if top_left[0]<=x<=bot_right[0] and top_left[1]<=y<=bot_right[1] else np.zeros(3)

class FF_Model(Model):
  def step(self, N):
    shapes = [
      rect((0, 0), (N, N/3), [0,0,1]),
      rect((0, N/3), (N, 2*N/3), [1,1,1]),
      rect((0, 2*N/3), (N, N), [1,0,0])
    ]
    self.grid = SingleGrid(N, N, True)
    self.schedule = RandomActivation(self)
    for x in range(N):
      for y in range(N):
        a = Cell(self, (x,y), shapes)
        self.grid.place_agent(a, (x, y))
        self.schedule.add(a)
    self.schedule.step()
      
class Cell(Agent):
  def __init__(self, model, n, shapes):
    super().__init__(n, model)
    self.shapes = shapes
    self.x, self.y = n
    self.color = np.zeros(3)
  def step(self):
    self.color = sum([s(self.x,self.y) for s in self.shapes], self.color)

The Mr. Potato Head Mesa model (see example reference above). Note that this uses the same Mesa Cell agent model as the French Flag example, further demonstrating that it's just the shape constraints ("body plan") that differ between the models.

In [29]:
def circle(center, radius, rgb):
  return lambda x, y: rgb if (math.sqrt((center[0]-x)**2 + (center[1]-y)**2) <= radius) else np.zeros(3)

def eye(center, eye_size, eye_color):
  return lambda x, y: sum([circle(center, eye_size[0], eye_color[0])(x,y), circle(center, eye_size[1], eye_color[1])(x,y)], np.zeros(3))

class MR_PH_Model(Model):
  def step(self, N):
    left_eye_center = (0.25*N,0.25*N) # x,y
    right_eye_center = (0.25*N,0.75*N) # x,y
    eye_size = (N/10, N/25) # radius outer, inner
    eye_color = ([1,0,0], [0,1,0]) # rgb outer, inner
    nose_tl = (0.35*N,0.4*N)
    nose_br = (0.7*N,0.6*N)
    mouth_tl = (0.8*N,0.2*N)
    mouth_br = (0.85*N,0.8*N)
    shapes = [
      eye(left_eye_center, eye_size, eye_color),
      eye(right_eye_center, eye_size, eye_color),
      rect(nose_tl, nose_br, [0,0,1]),
      rect(mouth_tl, mouth_br, [0,1,1])
    ]
    self.grid = SingleGrid(N, N, True)
    self.schedule = RandomActivation(self)
    for x in range(N):
      for y in range(N):
        a = Cell(self, (x,y), shapes)
        self.grid.place_agent(a, (x, y))
        self.schedule.add(a)
    self.schedule.step()


The interactive UI below allows you to choose the model and the min/max image sizes and it will simulate the shapes growing from min size to max size. A better model would actually do the work incrementally, but this is just a conceptual framework. Click the play button to start the animation.

In [30]:
models = {'French Flag':FF_Model(), 'Mr. Potato Head':MR_PH_Model()}

@interact(N1=(0, 500, 50), N2=(0, 500, 50), M=models.keys())
def run(N1=100, N2=300, M='Mr. Potato Head'):
  model = models[M]
  stepsize = 50
  fig, ax = plt.subplots(figsize=(N1/50, N1/50))

  # animation function. This is called sequentially  
  def animate(i):
    n = N1+i*stepsize
    model.step(n)
    fig.set_size_inches(n/stepsize, n/stepsize)
    im = np.zeros((n,n,3), dtype=float)
    for agent, x, y in model.grid.coord_iter():
      im[x,y] = agent.color
    ax.imshow(im, aspect='auto')
    return (fig,)
    
  anim = animation.FuncAnimation(fig, animate, frames=int((N2-N1)/stepsize)+1, interval=2000, blit=True)
  plt.close()
  rc('animation', html='jshtml')
  return anim


interactive(children=(IntSlider(value=100, description='N1', max=500, step=50), IntSlider(value=300, descripti…